In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/LLM Dataset.csv", encoding="latin1")
df.head()


,Patient_ID,Current_Age,Gender,Residence_Type,Education_Level,Monthly_Household_Income,Patient Report,Travel_Time_to_Hospital (min),School/Work_Impact,Age_at_Diagnosis,...,MCH (pg),RDW (%),Reticulocyte_Count (%),HbF_Percent (%),Transfusion_Frequency (units/month),Iron_Chelation_Therapy,Chelation_Adherence,BMI (kg/m²),Mental_Health_Status,Unnamed: 24
0,1,4,Male,Rural,NaN,"25,000",Iron overload,90,High,1.5,...,18.0,24,8.0,30.0,3.0,DFX + DFP,Low,14.2,Bad,NaN
1,2,12,Female,Urban,Primary,"25,000",Growth delay,40,Moderate,3.0,...,20.0,22,6.0,25.0,2.5,DFX,High,15.8,Bad,NaN
2,3,17,Male,Urban,Secondary,"30,000",No Report,25,Mild,6.0,...,23.0,18,3.0,10.0,1.0,DFP,Medium,19.5,Good,NaN
3,4,25,Female,Urban,Graduate,"45,000",Hypothyroid,10,NaN,8.0,...,25.0,14,2.0,5.0,0.5,DFX,High,20.8,Good,NaN
4,5,28,Male,Urban,Graduate,"55,000",No Report,1600,NaN,20.0,...,28.0,12,1.0,1.5,0.0,NaN,NaN,23.4,Good,NaN


In [ ]:
print(df)

     Patient_ID  Current_Age  Gender Residence_Type Education_Level  \
0             1            4    Male          Rural             NaN   
1             2           12  Female          Urban         Primary   
2             3           17    Male          Urban       Secondary   
3             4           25  Female          Urban        Graduate   
4             5           28    Male          Urban        Graduate   
..          ...          ...     ...            ...             ...   
612         613           41    Male          Urban        Graduate   
613         614           47  Female          Urban        Graduate   
614         615           24  Female          Urban       Secondary   
615         616           35    Male          Rural       Secondary   
616         617           48  Female          Urban        Graduate   

    Monthly_Household_Income Patient Report  Travel_Time_to_Hospital (min)  \
0                     25,000  Iron overload                          

In [ ]:
df1=df.drop(columns=['Unnamed: 24'])

In [ ]:
df1.columns.to_list()

['Patient_ID',
 'Current_Age',
 'Gender',
 'Residence_Type',
 'Education_Level',
 'Monthly_Household_Income',
 'Patient Report',
 'Travel_Time_to_Hospital (min)',
 'School/Work_Impact',
 'Age_at_Diagnosis',
 'Thalassemia_Type',
 'Splenectomy_Status',
 'Pre-Transfusion_Hb (g/dL)',
 'Serum_Ferritin (ng/mL)',
 'MCV (fL)',
 'MCH (pg)',
 'RDW (%)',
 'Reticulocyte_Count (%)',
 'HbF_Percent (%)',
 'Transfusion_Frequency (units/month)',
 'Iron_Chelation_Therapy',
 'Chelation_Adherence',
 'BMI (kg/m²)',
 'Mental_Health_Status']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categ_cols = ['Gender', 'Residence_Type', 'Education_Level',
              'School/Work_Impact','Thalassemia_Type','Splenectomy_Status',
              'Iron_Chelation_Therapy','Chelation_Adherence','Mental_Health_Status']

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_cats = encoder.fit_transform(df1[categ_cols])
encoded_cat_cols = encoder.get_feature_names_out(categ_cols)
encoded_df = pd.DataFrame(encoded_cats, columns=encoded_cat_cols, index=df1.index)
print(encoded_df)

     Gender_Male  Residence_Type_Urban  Education_Level_Postgraduate  \
0            1.0                   0.0                           0.0   
1            0.0                   1.0                           0.0   
2            1.0                   1.0                           0.0   
3            0.0                   1.0                           0.0   
4            1.0                   1.0                           0.0   
..           ...                   ...                           ...   
612          1.0                   1.0                           0.0   
613          0.0                   1.0                           0.0   
614          0.0                   1.0                           0.0   
615          1.0                   0.0                           0.0   
616          0.0                   1.0                           0.0   

     Education_Level_Preschool  Education_Level_Primary  \
0                          0.0                      0.0   
1                

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
texts = df1['Patient Report'].astype(str).tolist()
max_length = 64
embeddings = []
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    embeddings.append(cls_embedding)
embeddings_df = pd.DataFrame(embeddings, index=df1.index)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
df_tabular = df1.drop(columns=['Patient Report'])
final_df = pd.concat([encoded_df, embeddings_df], axis=1)
print("Final feature matrix shape:", final_df.shape)
print(final_df.head())


Final feature matrix shape: (617, 790)
   Gender_Male  Residence_Type_Urban  Education_Level_Postgraduate  \
0          1.0                   0.0                           0.0   
1          0.0                   1.0                           0.0   
2          1.0                   1.0                           0.0   
3          0.0                   1.0                           0.0   
4          1.0                   1.0                           0.0   

   Education_Level_Preschool  Education_Level_Primary  \
0                        0.0                      0.0   
1                        0.0                      1.0   
2                        0.0                      0.0   
3                        0.0                      0.0   
4                        0.0                      0.0   

   Education_Level_Secondary  Education_Level_nan  School/Work_Impact_Low  \
0                        0.0                  1.0                     0.0   
1                        0.0               

In [ ]:
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder
categ_cols = ['Gender', 'Residence_Type', 'Education_Level',
              'School/Work_Impact','Thalassemia_Type','Splenectomy_Status',
              'Iron_Chelation_Therapy','Chelation_Adherence']
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_cats = encoder.fit_transform(df1[categ_cols])
encoded_cat_cols = encoder.get_feature_names_out(categ_cols)
encoded_df = pd.DataFrame(encoded_cats, columns=encoded_cat_cols, index=df1.index)
embeddings_df_mean = embeddings_df.mean(axis=1).to_frame(name="Patient Report Embedding")
X = pd.concat([encoded_df, embeddings_df_mean], axis=1)
y = df1['Mental_Health_Status'].astype('category').cat.codes
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False
)
xgb_clf.fit(X, y)
explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer.shap_values(X)
prescription_map = {
    "School/Work_Impact": "Reduce workload or seek support from school/workplace to minimize stress.",
    "Age_at_Diagnosis": "Early diagnosis generally improves outcomes; since age cannot change, focus on strict adherence to treatment.",
    "Thalassemia_Type": "Cannot be changed, but different types need tailored care. Follow type-specific medical advice.",
    "Splenectomy_Status": "Splenectomy cannot be reversed; ensure vaccinations and infection prevention measures.",
    "Self-Reported_Comorbidities": "Manage comorbidities (e.g., diabetes, heart disease) with regular follow-up and lifestyle changes.",
    "Pre-Transfusion_Hb (g/dL)": "Maintain hemoglobin closer to target (9–10.5 g/dL) by optimizing transfusion schedule.",
    "Serum_Ferritin (ng/mL)": "Lower iron overload with strict adherence to chelation therapy and regular ferritin monitoring.",
    "MCV (fL)": "MCV abnormalities reflect red cell health; maintain adequate transfusions and monitor closely.",
    "MCH (pg)": "Improve red blood cell indices with adequate transfusion and follow nutrition advice.",
    "RDW (%)": "Reduce variability in red cell size by maintaining steady transfusion and iron chelation.",
    "Reticulocyte_Count (%)": "Monitor bone marrow activity; abnormal counts may need treatment adjustments.",
    "HbF_Percent (%)": "Higher HbF can help; some therapies (like hydroxyurea) may be considered under medical advice.",
    "Transfusion_Frequency (units/month)": "Adjust transfusion frequency to prevent anemia while avoiding overload.",
    "Iron_Chelation_Therapy": "Ensure consistent chelation therapy to reduce iron overload complications.",
    "Chelation_Adherence": "Improve adherence to chelation therapy through reminders, counseling, or simplified schedules.",
    "Patient Report Embedding": "Seek psychological support, counseling, and mental health monitoring.",
    "BMI (kg/m²)": "Maintain healthy BMI with balanced diet and regular physical activity."
}
def get_base_feature(feat):
    if "_" in feat:
        return "_".join(feat.split("_")[:-1])
    return feat
def explain_patient_with_prescription(patient_id):
    if patient_id not in df1['Patient_ID'].values:
        print(f" Patient_ID {patient_id} not found in the dataset.")
        return

    idx = df1.index[df1['Patient_ID'] == patient_id][0]
    patient_X = X.iloc[idx:idx+1]
    pred_class = xgb_clf.predict(patient_X)[0]
    class_name = df1['Mental_Health_Status'].astype('category').cat.categories[pred_class]
    print(f"\n Predicted Mental Health Status for Patient {patient_id}: {class_name}")
    shap_vals = shap_values[pred_class][idx] if isinstance(shap_values, list) else shap_values[idx]

    shap_df = pd.DataFrame({
        "feature": patient_X.columns,
        "value": patient_X.values.flatten(),
        "shap_value": shap_vals
    })
    shap_df = shap_df.reindex(shap_df.shap_value.abs().sort_values(ascending=False).index)
    top3 = shap_df.head(3)
    print("\nTop 3 factors influencing this prediction:")
    for _, row in top3.iterrows():
        direction = " positive" if row["shap_value"] > 0 else " negative"
        print(f"- {row['feature']} = {row['value']} ({direction} impact)")
    if class_name.lower() == "bad":
        print("\n Data-Driven Prescriptions:")
        for _, row in top3.iterrows():
            feat_base = get_base_feature(row['feature'])
            if feat_base in prescription_map:
                print(f"- {feat_base}: {prescription_map[feat_base]}")

    return class_name, top3
pid = int(input("Enter Patient_ID to check mental health status: "))
explain_patient_with_prescription(pid)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [02:09:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Enter Patient_ID to check mental health status: 2

 Predicted Mental Health Status for Patient 2: Bad

Top 3 factors influencing this prediction:
- School/Work_Impact_nan = 0.0 (↓ negative impact)
- Patient Report Embedding = -0.006348683964461088 (↓ negative impact)
- Thalassemia_Type_Major = 1.0 (↓ negative impact)

 Data-Driven Prescriptions:
- School/Work_Impact: Reduce workload or seek support from school/workplace to minimize stress.
- Patient Report Embedding: Seek psychological support, counseling, and mental health monitoring.
- Thalassemia_Type: Cannot be changed, but different types need tailored care. Follow type-specific medical advice.


('Bad',
                      feature     value  shap_value
 10    School/Work_Impact_nan  0.000000   -3.187432
 21  Patient Report Embedding -0.006349   -1.885620
 11    Thalassemia_Type_Major  1.000000   -0.940818)

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import shap

categ_cols = ['Gender', 'Residence_Type', 'Education_Level',
              'School/Work_Impact','Thalassemia_Type','Splenectomy_Status',
              'Iron_Chelation_Therapy','Chelation_Adherence']

encoder = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")
encoder.fit(df1[categ_cols])
encoded_feature_names = encoder.get_feature_names_out(categ_cols)

def validate_choice(prompt, valid_options):
    choice = input(prompt).strip().replace("'", "").replace('"', "")
    while choice.upper() not in [opt.upper() for opt in valid_options]:
        print(f"Invalid choice. Please enter one of: {valid_options}")
        choice = input(prompt).strip().replace("'", "").replace('"', "")
    for opt in valid_options:
        if choice.upper() == opt.upper():
            return opt

def validate_numeric(prompt, type_func=float):
    while True:
        try:
            return type_func(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

def get_base_feature(feature_name):
    return feature_name.split("_")[0]

def get_patient_input():
    patient_data = {}
    patient_data['Current_Age'] = validate_numeric("Enter Current Age: ", int)
    patient_data['Monthly_Household_Income'] = validate_numeric("Enter Monthly Household Income: ", int)
    patient_data['Age_at_Diagnosis'] = validate_numeric("Enter Age at Diagnosis: ", int)
    patient_data['Pre-Transfusion_Hb (g/dL)'] = validate_numeric("Enter Pre-Transfusion Hb (g/dL): ")
    patient_data['Serum_Ferritin (ng/mL)'] = validate_numeric("Enter Serum Ferritin (ng/mL): ")
    patient_data['MCV (fL)'] = validate_numeric("Enter MCV (fL): ")
    patient_data['MCH (pg)'] = validate_numeric("Enter MCH (pg): ")
    patient_data['RDW (%)'] = validate_numeric("Enter RDW (%): ")
    patient_data['Reticulocyte_Count (%)'] = validate_numeric("Enter Reticulocyte Count (%): ")
    patient_data['HbF_Percent (%)'] = validate_numeric("Enter HbF Percent (%): ")
    patient_data['Transfusion_Frequency (units/month)'] = validate_numeric("Enter Transfusion Frequency (units/month): ", int)
    patient_data['BMI (kg/m²)'] = validate_numeric("Enter BMI (kg/m²): ")
    patient_data['Patient Report'] = input("Enter Patient Report (symptoms/notes): ")
    patient_data['Gender'] = validate_choice("Enter Gender (Male/Female): ", ["Male", "Female"])
    patient_data['Residence_Type'] = validate_choice("Enter Residence Type (Urban/Rural): ", ["Urban", "Rural"])
    patient_data['Education_Level'] = validate_choice("Enter Education Level (Preschool/Primary/Secondary/Graduate/Postgraduate): ", ["Preschool", "Primary", "Secondary", "Graduate", "Postgraduate"])
    patient_data['School/Work_Impact'] = validate_choice("Enter School/Work Impact (Low/Mild/Moderate/High): ", ["Low", "Mild", "Moderate", "High"])
    patient_data['Thalassemia_Type'] = validate_choice("Enter Thalassemia Type (Minor/Intermedia/Major): ", ["Minor", "Intermedia", "Major"])
    patient_data['Splenectomy_Status'] = validate_choice("Enter Splenectomy Status (Yes/No): ", ["Yes", "No"])
    patient_data['Iron_Chelation_Therapy'] = validate_choice("Enter Iron Chelation Therapy (DFO/DFX/DFP/None): ", ["DFO", "DFX", "DFP", "None"])
    patient_data['Chelation_Adherence'] = validate_choice("Enter Chelation Adherence (High/Medium/Low): ", ["High", "Medium", "Low"])
    return patient_data

def predict_new_patient(new_patient):
    new_df = pd.DataFrame([new_patient])
    encoded_cats = encoder.transform(new_df[categ_cols])
    encoded_df_new = pd.DataFrame(encoded_cats, columns=encoded_feature_names, index=new_df.index)
    with torch.no_grad():
        inputs = tokenizer(new_df['Patient Report'].astype(str).values[0],
                           return_tensors="pt", truncation=True,
                           padding="max_length", max_length=64)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    embedding_df_new = pd.DataFrame([cls_embedding], index=new_df.index)
    embedding_mean = embedding_df_new.mean(axis=1).to_frame(name="Patient Report Embedding")
    X_new = pd.concat([encoded_df_new, embedding_mean], axis=1)
    pred_class_idx = xgb_clf.predict(X_new)[0]
    class_name = df1['Mental_Health_Status'].astype('category').cat.categories[pred_class_idx]
    shap_vals = shap.TreeExplainer(xgb_clf).shap_values(X_new)
    shap_df = pd.DataFrame({
        "feature": X_new.columns,
        "value": X_new.values.flatten(),
        "shap_value": shap_vals[0] if isinstance(shap_vals, list) else shap_vals.flatten()
    })
    shap_df = shap_df.reindex(shap_df.shap_value.abs().sort_values(ascending=False).index)
    top3 = shap_df.head(3)
    print(f"\nPredicted Mental Health Status: {class_name}")
    print("\nTop 3 factors influencing this prediction:")
    for _, row in top3.iterrows():
        direction = "positive" if row["shap_value"] > 0 else "negative"
        print(f"- {row['feature']} = {row['value']} ({direction} impact)")
    if class_name.lower() == "bad":
        print("\nData-Driven Prescriptions:")
        for _, row in top3.iterrows():
            feat_base = get_base_feature(row['feature'])
            if feat_base in prescription_map:
                print(f"- {feat_base}: {prescription_map[feat_base]}")
    return class_name, top3

print("Enter New Patient Details Below:\n")
new_patient = get_patient_input()
pred_class, top_features = predict_new_patient(new_patient)


Enter New Patient Details Below:

Enter Current Age: 12
Enter Monthly Household Income: 2300
Enter Age at Diagnosis: 4
Enter Pre-Transfusion Hb (g/dL): 5
Enter Serum Ferritin (ng/mL): 2800
Enter MCV (fL): 6
Enter MCH (pg): 4
Enter RDW (%): 8
Enter Reticulocyte Count (%): 9
Enter HbF Percent (%): 3
Enter Transfusion Frequency (units/month): 8
Enter BMI (kg/m²): 14
Enter Patient Report (symptoms/notes): Growth delay
Enter Gender (Male/Female): Male
Enter Residence Type (Urban/Rural): Urban
Enter Education Level (Preschool/Primary/Secondary/Graduate/Postgraduate): Primary
Enter School/Work Impact (Low/Mild/Moderate/High): Low
Enter Thalassemia Type (Minor/Intermedia/Major): Minor
Enter Splenectomy Status (Yes/No): Yes
Enter Iron Chelation Therapy (DFO/DFX/DFP/None): DFO
Enter Chelation Adherence (High/Medium/Low): High


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Predicted Mental Health Status: Bad

Top 3 factors influencing this prediction:
- School/Work_Impact_nan = 0.0 (negative impact)
- Splenectomy_Status_Yes = 1.0 (negative impact)
- Patient Report Embedding = -0.006348683964461088 (negative impact)

Data-Driven Prescriptions:
- Splenectomy: Follow-up and monitor for infections.
